<a href="https://colab.research.google.com/github/nico-ig/RockPaintingDetector/blob/main/Faster_R_CNN_Model_to_Detect_Rock_Painting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single Class

In [ ]:
import os

project_name = "CocoSingleClass"
root_dir = "RockPaintingDetector"
local_path = "/content"

# Training version
version = str(3)

# Names for dataset splits (train, val, test) including version
train_instance = "rockart_train_v" + version
val_instance = "rockart_val_v" + version
test_instance = "rockart_test_v" + version

# Set Paths
project_path = os.path.join(local_path, root_dir, project_name)
dataset_path = os.path.join(local_path, root_dir, 'dataset', 'single-class', 'coco', 'coco')

In [ ]:
# Register COCO datasets in Detectron2 for training, validation, and testing
from detectron2.data.datasets import register_coco_instances

register_coco_instances(
    train_instance, {},  # instance name and empty metadata
    f"{dataset_path}/train/_annotations.coco.json",  # COCO annotation file
    f"{dataset_path}/train"  # folder with training images
)

register_coco_instances(
    val_instance, {},
    f"{dataset_path}/valid/_annotations.coco.json",
    f"{dataset_path}/valid"
)

## Training

In [ ]:
import os
import json

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.events import EventStorage

# Load base config for the pretrained Faster R-CNN COCO model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Define registered train and validation datasets
cfg.DATASETS.TRAIN = (train_instance,)
cfg.DATASETS.TEST = (val_instance,)
cfg.DATALOADER.NUM_WORKERS = 2

# Use pretrained COCO weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

# Configure training parameters
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 3000
cfg.SOLVER.STEPS = (2000, 2700)
cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128

# Set number of classes based on COCO dataset annotations
with open(dataset_path + "/train/_annotations.coco.json") as f:
    data = json.load(f)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(data["categories"])

# Create output directory for checkpoints and logs
cfg.OUTPUT_DIR = os.path.join(project_path, 'coco', 'output')
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Initialize and start training
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

# Dual Class

## Setup

In [ ]:
import os

project_name = "CocoDualClass"
root_dir = "RockPaintingDetector"
local_path = "/content"

# Training version
version = str(1)

# Names for dataset splits (train, val, test) including version
train_instance = "rockart_train_dual_v" + version
val_instance = "rockart_val_dual_v" + version
test_instance = "rockart_test_dual_v" + version

# Set Paths
project_path = os.path.join(local_path, root_dir, project_name)
dataset_path = os.path.join(local_path, root_dir, 'dataset', 'dual-class', 'coco', 'coco')

In [ ]:
# Register COCO datasets in Detectron2 for training, validation, and testing
from detectron2.data.datasets import register_coco_instances

register_coco_instances(
    train_instance, {},  # instance name and empty metadata
    f"{dataset_path}/train/_annotations.coco.json",  # COCO annotation file
    f"{dataset_path}/train"  # folder with training images
)

register_coco_instances(
    val_instance, {},
    f"{dataset_path}/valid/_annotations.coco.json",
    f"{dataset_path}/valid"
)

## Training

In [ ]:
import os
import json

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.events import EventStorage

# Load base config for the pretrained Faster R-CNN COCO model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Define registered train and validation datasets
cfg.DATASETS.TRAIN = (train_instance,)
cfg.DATASETS.TEST = (val_instance,)
cfg.DATALOADER.NUM_WORKERS = 2

# Use single class weights
cfg.MODEL.WEIGHTS = os.path.join(local_path, root_dir, 'CocoSingleClass', 'coco', 'output', 'model_final.pth')

# Configure training parameters
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 1e-4
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1200, 1800)
cfg.SOLVER.GAMMA = 0.1
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128

# Set number of classes based on COCO dataset annotations
with open(dataset_path + "/train/_annotations.coco.json") as f:
    data = json.load(f)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(data["categories"])

# Create output directory for checkpoints and logs
cfg.OUTPUT_DIR = os.path.join(project_path, 'coco', 'output')
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Initialize and start training
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()